# Fuzzy Logic Clustering

A common problem for database administrators is that of grouping unique entities when the integrity of the records is questionable. In other words, names might be missing or mispelled, records might be incomplete but still individually identifiable using human intuition. With tens of thousands of records to identity, human intuition must be supported with additional technical muscle. One approach to this problem is to apply Levenshtein distance across the fields of two records to determine if they are related. Groups will be formed by similarity and weights will be applied to the data fields according to their value to us (e.g. SNN is more identifiable than suffix or zip code).

## <a name="TOC"></a> Table of Contents:
---
1. [Data Set](#data)
2. [Description of Algorithm](#description)
  1. [Card Analogy](#cards)
  2. [Detailed Description](#detailed)
3. [Proof of Process](#proof)
4. [Development of Methods](#methods)
5. [Full Algorithm](#algorithm)
  1. [Instantiation](#instantiate)
  2. [Iteration](#iterate)
  3. [Pull Group Records](#pull)


In [1]:
# -------------------- CONFIGURE ENVIRONMENT -------------------- #

# Environment hard reset
%reset -f

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Standard math libraries
import math as math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rand
%matplotlib inline

# SQL-similar commands
from collections import Counter

# System level control
import sys

# Support for progress bar
import time
from IPython.display import clear_output

# Levenshtein fuzzy comparisons
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

# Configure paths
from pathlib import Path
data_path = Path('Data')
fig_path = Path('PyFigures')

# Setup log file
f = open('Logs.txt', 'w').close()
filename = 'Logs.txt'


## <a name="data"></a> [Data Set](#TOC)
---

The synthesized data that we will be studying comes in the form of records. I have simulated data fields with names, addresses and SSNs. There are multiple groups within this dataset and therefore multiple groups should be identified by our algorithm. Many of these identities are one to one such that a single identity has a single record but several identities are described by several records in a one to many scenario. It is my hope that my algorithm will successfully identify these relationships.


In [2]:
# -------------------- IMPORT DATA -------------------- #

# Import from CSV
Data = pd.read_csv(data_path / 'SyntheticData.csv')

# See all data
Data.head(20)


,FIRST,MIDDLE,LAST,ADDRESS,CITY,STATE,ZIP,SSN
0,HARVEY,B,MILK,1930 RECRUIT YOU AVE,SAN FRANCISCO,CA,18896.0,314159265.0
1,HARVEY,B,MILK,1930 RECRUIT YOU AVE,SF,CA,18896.0,314159265.0
2,H,B,MLLK,1930 RECRUIT YOU AVE,SF,CA,NaN,314159265.0
3,H,NaN,NaN,1930 RECRUIT U,NaN,NaN,18896.0,NaN
4,RICK,P,SANCHEZ,EARTH C-137,TACOMA,WA,98402.0,929601596.0
5,PYOTR,I,TCHAIKOVSKY,1840 N ST PETER,VOTKINSK,GA,11893.0,155633999.0
6,RUPERT,L,DONLEY,518 EVEREST BLVD,UDEMY,FL,12358.0,123789235.0
7,ELEANOR,P,TWINE,7878 SEVEN EIGHT,SMALL TOWN,LA,26916.0,148978105.0
8,MICHELL,M,MENENDEZ,58 CLOUDLESS SKY WAY,NaN,NaN,32156.0,203254687.0
9,CHARLES,F,KANE,794 BROKEN HOME AVE,ROSEBUD,PA,56465.0,489324789.0


In [3]:
# -------------------- DATA PREPROCESSING -------------------- #

# Replace missing with NaN object
Data.fillna(np.nan)

# Make sure NaNs are identifiable
math.isnan( Data.iloc[3]['MIDDLE'] )


True

## <a name="description"></a> [Description of Algorithm](#TOC)
---


### <a name="cards"></a> [Card Analogy](#description)

My process for clustering will follow a standard card analogy. Imagine that we have a deck of cards and we wish to cluster them. By what? Suit? Royalty? Value? We say that we want to sort by suit but that the symbols for the suite have been obscured and disfigured. We know that hearts and diamonds, clubs and spades look similar but that there is a way to compare their similarity. Our process follows:

0. Throw one card on the table and make it a group

1. Select the next card

2. Compare suits
  1. If the new suit is sufficiently similar to an existing group then it joins the group that it is most similar to
  2. If the new suit it not sufficiently similar to any existing group then it becomes its own group
  
3. Are there more cards in the deck?
  1. Yes, go to 1.
  2. No, end.

This process ensures that a new group can only be formed by failing to belong to an existing group.


### <a name="detailed"></a> [Detailed Description](#description)

#### Weighted Average

Obviously, the card analogy does not entirely apply to the specifics of this problem. We have records with features (entries for a column) to identify them, some of which matter more to us than others for the purpose of identification.

$$
R_n = \{ \; f_0, f_1, f_N \; \}
$$

We express our system of priority by assigning weights to each feature by order of significance, like so:

$$
\vec{ \omega } = \omega_0 + \omega_1 + \omega_N = \sum_{n = 0}^{N} \omega_n = 1.0
$$

Individual features of a record can be compared to the features of another record using Levenshtein distance. A ratio can be obtained which refects the percentage of similarity between the features. To express the similarity between the records however, we must apply the weights to the individual feature similarities. We can find the similarity $\text{s}$:

$$
R_a = \{ \; f_{a0}, f_{a1}, ... \; f_{aN} \; \} \\
R_b = \{ \; f_{b0}, f_{b1}, ... \; f_{bN} \; \} \\
$$

$$
\text{s} = \omega_0 \cdot \text{Lev}( f_{a0}, f_{b0} ) \; + \; \omega_1 \cdot \text{Lev}( f_{a1}, f_{b1} ) \; + \; ... \; \omega_N \cdot \text{Lev}( f_{aN}, f_{bN} )
$$

If $R_a$ is an existing group unto itself and $\text{s}$ is greater than some threshhold big $\text{S}$ then $R_b$ joins the group for $R_a$. Then a new record $R_c$ is compared to the center of $R_a$ (more on this later). The process repeats.

However, if $\text{s} < \text{S}$, then $R_b$ becomes its own group. Then record $R_c$ is compared to records $R_a$ and $R_b$ and the process repeats.

#### Weight Adjustment

If a record has missing features, then the algorithm suffers a statistical nightmare. For example, imagine that we have two records $R_a$ and $R_b$ with four features which are equal in all respects except for one: $f_{a2} = \text{NaN} \neq f_{b2}$. If all weights are adjusted equally, then the best similarity that these records could obtain would be a 75%, which might be too low to exceed the threshhold. If we want these records to be equated, then we must either lower our standards of adjust the weights, but how?

When a feature is missing, the algorithm drops the feature from the comparison and adjusts the weights while preserving the biasing. Say that feature number 2 is missing, then let $\vec{ \omega_{A} }$ represent the adjusted weights:

$$
\text{Let} \; \vec{ \omega } = [ \omega_0, \omega_1, \omega_2, \omega_3 ] \; \text{s.t.} \; \sum_{n = 0}^{N} \omega_n = 1.0
$$

$$
\vec{ \omega_{A} } = [ \omega_0, \omega_1, 0, \omega_3 ] \longrightarrow [ \frac{\omega_0}{\omega_0 + \omega_1+ \omega_3}, \frac{\omega_1}{\omega_0 + \omega_1+ \omega_3}, 0, \frac{\omega_3}{\omega_0 + \omega_1+ \omega_3} ]
$$

Using numeric values, let's say that $\vec{ \omega } = [ 0.20, 0.20, 0.50, 0.10 ]$ with feature number 2 missing, then:

$$
\vec{ \omega_{A} } = [ 0.20, 0.20, 0, 0.10 ] \longrightarrow [ \frac{0.20}{0.20 + 0.20 + 0.10}, \frac{0.20}{0.20 + 0.20 + 0.10}, 0, \frac{0.10}{0.20 + 0.20 + 0.10} ] = [ 0.40, 0.40, 0.20 ]
$$

The first and second weights maintain their proportionality to the final weight. Our algorithm has adjusted without loosing its assigned biases.

#### The Most Human Human

One issue facing our algorithm lies in the representation of groups. Storing groups programmatically is an issue unto itself, but comparing individual records against a group is the key challenge. With numeric data, there exists an idea of the centroid: an n-dimensional center of mass for a set of records. With text data, however, the centroid becomes more abstract.

There are two approaches for finding the best representative of a group identity:

1. Completeness - The record with the fewest missing fields.
2. Similarity - The record that is the most common to the other records in the group.

We will use approach 1 for groups with 2 or fewer records and approach 2 for groups with three or more records.

#### Representing Hierarchy

Another issue facing this system is how to store the hierarchy. My solution to this problem is to have a separate table with a column for group number, record number, and a flag for the centroid. Then we can quickly select a group number, obtain its records, and query the main dataframe for the record level details. We can manage the hierarchy efficiently without needing to pull the heavier records through the query, this approach should lower the computation requirements.


## <a name="proof"></a> [Proof of Process](#TOC)
---

Prior to developing sets of functions to execute my proposed algorithm, I want to develop the process to prove that it works in its raw form. Once I have established the viability of this route I can proceed to the next stage.


In [4]:
# -------------------- SETUP WEIGHTS -------------------- #

# Set biasing
Omega = [ 0.18, 0.15, 0.20, 0.05, 0.05, 0.10, 0.02, 0.25 ]

# Check that weights balance
if np.sum( Omega ) != 1:
    print('Weights are not in balance.')

# Select two records
#rec_ind = range( 0, len(data) )
rec_ind = [ 1, 7 ]

# -------------------- MISSING FEATURES -------------------- #

# Check for null in each record
for i in range( 0, len(rec_ind) ):
    
    # Find true indexes
    true_ind = np.where( Data.iloc[ rec_ind[i] ].isnull().values == True )

    # Assign status
    status = 'Missing {}'.format( *true_ind )
    
    # Print record null status
    print("R{} \t: {}".format(rec_ind[i], status) )
    
# Add a blank line
print('')
    
# -------------------- ADJUST WEIGHTS -------------------- #

# Copy and print normal weights
Omega_A = Omega.copy()
print('Original: \t{}'.format( Omega ) )

# Zero out missing indexes
for i in true_ind[0]:
    Omega_A[ i ] = 0
    
# Divide the weights by the sum of the non-zero weights, print results
Omega_A = Omega_A / round( np.sum( [ Omega_A[i] for i in np.nonzero( Omega_A )[0] ] ), 20 )
print('Adjusted: \t{}'.format( Omega_A ))

# Check that the weights balance out
if not math.isclose( np.sum( Omega_A ), 1, abs_tol = 1e-100 ):
    print('Weights are not in balance.')


R1 	: Missing []
R7 	: Missing []

Original: 	[0.18, 0.15, 0.2, 0.05, 0.05, 0.1, 0.02, 0.25]
Adjusted: 	[0.18 0.15 0.2  0.05 0.05 0.1  0.02 0.25]


So the weights are adjusting properly and the proportionality is preserved but now we must run the cummulative product of the Levenshtein distance and the adjusted weights to determine which records are similar to which records.


In [5]:
# -------------------- RUN SIMILARITY -------------------- #

# Set big S
S = 75

# Instantiate s
s = np.array([])

# Run comparison
for j in range(0, len(Data.columns)):
    s = np.append(s, fuzz.ratio( Data.iloc[ rec_ind[0] ].apply(str)[j], Data.iloc[ rec_ind[1] ].apply(str)[j] ) )
    
# Print LEV({}) results
print( 'Lev Ratios: {}'.format(s) )

# Linear product with adjusted weights
s = np.inner( s, Omega_A )
print( 'Similarity: {}'.format(s) )


Lev Ratios: [31.  0. 22. 22. 17. 50. 57. 55.]
Similarity: 31.82


We can see that the missing feature has scored a zero on the similarity, identical records have been assigned a perfect Levenshtein ratio, and non-identical records have scored somewhere in between. This is as expected and the inner product of the ratios with the weights yields the similarity score that we have been looking for. Now we must decide if the individual record belongs with the group representative (centroid) or belongs as a record unto itself.

We will pretend, for simplicity sake, that the first record in the comparison was a group centroid and that it was in the Groups dataframe the whole time as the only existing group.


In [6]:
# -------------------- DETERMINE GROUPING -------------------- #

# Create Groups dataframe template
grp_col = [ 'Group Number', 'Record Number', 'Centroid Flag' ]
#Groups = pd.DataFrame( columns = grp_col )

# Add the first record and pretend it was a group leader this whole time
Groups = pd.DataFrame( data = { grp_col[0] : 0, grp_col[1] : rec_ind[0], grp_col[2] : True }, index = {0} )

# If the record belong in the group
if s >= S:
    # Find group number for the record being tested
    grp_no = Groups.loc[ Groups[ Groups.columns[1] ] == rec_ind[0] ][ Groups.columns[0] ].values[0]
    
    # Save record number under consideration
    rec_no = rec_ind[1]
    
    # Create temp record for append
    Record = pd.DataFrame(data = { grp_col[0] : grp_no, grp_col[1] : rec_no, grp_col[2] : False }, index = {0})
    Groups = Groups.append( Record, ignore_index = True )

# Otherwise make the record a new group
else:
    # Assign a new group number
    grp_no = Groups[ Groups.columns[0] ].values.max() + 1
    
    # Save record number 
    rec_no = rec_ind[1]
    
    # Create temp record for append
    Record = pd.DataFrame(data = { grp_col[0] : grp_no, grp_col[1] : rec_no, grp_col[2] : True }, index = {0})
    Groups = Groups.append( Record, ignore_index = True )

# Print results
Groups


,Group Number,Record Number,Centroid Flag
0,0,1,True
1,1,7,True


Excellent, I have tested cases for high and low similarity and the resulting group logic is as expected. I can confidently claim that the system is working and that the concept has been proven valid.


## <a name="methods"></a> [Development of Methods](#TOC)
---

Now that the concept behind the algorithm has been validated, I can break the algorithm into discrete functions to improve the efficiency and safety of the process. These functions can be independently validated and ultimately sewn together for the final algorithm.

### <a name="TOM"></a> Table of Methods:
---
* [Iterator](#iterator) - Returns (opt-destructive) a record from a dataframe as a series.
* [Adjust Weights](#adjust_weights) - Returns adjusted weights for two specific records.
* [Centroid Iterator](#centroid_iterator) - Returns centroid record for a specific group number.
* [Score Similarity](#score_similarity) - Returns the similarity between two records. 
* [Score Against Groups](#score_against_groups) - Returns the similarities between a record and each centroid.
* [Centroid Assessment](#centroid_assessment) - Evaluates existing groups to elect new centroids.
* [Append to Groups](#append_to_groups) - Adds a record to the groups dataframe.
* [Progress Bar](#progress) - Functions to support the progress bar.


In [7]:
# ----- SET GLOBAL VARS ----- #

# Omega bias preferences
Omega = [ 0.18, 0.15, 0.20, 0.05, 0.05, 0.10, 0.02, 0.25 ]

# Set big S
S = 75

# ----- SETUP LOGS ----- #

filename = 'Logs.txt'
logs = open(filename, 'w').close()
logs = open(filename, 'a')

# ----- COPY OF DATASET ----- #

Dataset = Data.copy()

# ----- SIMULATE GROUPS ----- #

# Fake data
fake_groups = [ 0, 0, 1, 2, 3, 4, 4, 4, 5, 6, 6 ]
fake_record = [ 0, 1, 2, 3, 4, 13, 14, 15, 12, 16, 17 ]
fake_roles  = [ True, False, True, True, True, True, False, False, True, False, True ]

# Template columns
grp_col = [ 'GROUP NUMBER', 'RECORD NUMBER', 'CENTROID FLAG' ]

# Form data for simulation
Groups = pd.DataFrame( np.transpose( np.array([ fake_groups, fake_record, fake_roles ]) ), columns = grp_col )
Groups.head(15)


,GROUP NUMBER,RECORD NUMBER,CENTROID FLAG
0,0,0,1
1,0,1,0
2,1,2,1
3,2,3,1
4,3,4,1
5,4,13,1
6,4,14,0
7,4,15,0
8,5,12,1
9,6,16,0


### <a name="iterator"></a> [Iterator](#methods)

This function operates as an iterator for any data frame. By calling this function, we pop a record from the dataframe and return it as a series.


In [8]:
# ----- ITERATE RECORDS FROM THE DECK ----- #

def iterator( df, destruct, verb ):
    """Returns (opt-destructive) a record from a dataframe as a series."""
    
    # Verbosity and logs
    if verb:
        print('Iterating.')
    
    # Copy the last record in the deck
    try:
        next_record = df.iloc[ len(df)-1 ].copy()
    except:
        print('ERROR: ', sys.exc_info()[0], 'in iterate.')
    
    # Pop the last record from the deck
    if destruct: df.drop( df.index[[next_record.name]], inplace = True )
    
    # Return next record
    return next_record
    
# <----- DEMO -----> #

# # Print a card using iterate
# card = iterator( Dataset, False, False )
# print( '{}: \t{} {}'.format(i, card.FIRST, card.LAST) )


### <a name="adjust_weights"></a> [Adjust Weights](#methods)

Here we take in the records for comparison and determine what weights must be dropped from the normal weights to insure that the bias is conserved but that the missing values do not eliminate the possibility for a match.


In [9]:
# ----- ADJUST WEIGHTS FOR MISSING VALUES ----- #

def adjust_weights( Norm, RA, RB, verb ):
    """Returns adjusted weights for two specific records."""
    
    # ----- MISSING FEATURES ----- #

    # Check for null in each record

    # Find true indexes
    missing_ind_RA = list(np.where( RA.isnull().values == True )[0])
    if missing_ind_RA: 
        if verb:
            print('RA #{} \t: Missing {}'.format( RA.name, missing_ind_RA ) )

    # Find missing indexes within Group
    missing_ind_RB = list(np.where( RB.isnull().values == True )[0])
    if missing_ind_RB:
        if verb:
            print('Record #{} \t: Missing {}'.format( RB.name, missing_ind_RB ) )

    # Combine lists of missing indexes
    missing_ind = missing_ind_RA + missing_ind_RB

    # Copy normal weights
    Omega_A = Norm.copy()
        
    # ----- ADJUSTMENT ----- #

    # Zero out missing indexes
    for i in missing_ind:
        Omega_A[ i ] = 0
        
    # Speak if adjusting weights
    if missing_ind:
        if verb:
            print('Adjusting weights.')
            
    # Divide the weights by the sum of the non-zero weights, print results
    Omega_A = list(Omega_A / round( np.sum( [ Omega_A[i] for i in np.nonzero( Omega_A )[0] ] ), 20 ))
    
    # Speak if adjusting weights
    if missing_ind:
        if verb:
            print('Omega_A : {}'.format(Omega_A) )
            
    # Check that the weights balance out
    if not math.isclose( np.sum( Omega_A ), 1, abs_tol = 1e-100 ):
        if verb:
            print('Weights are not in balance.')
        return -1

    # Close logs
    logs.close()
    
    # Return next record
    return Omega_A

# <----- DEMO -----> #

# RA = Dataset.iloc[7]
# RB = Dataset.iloc[8]
# Omega_A = adjust_weights( Omega, RA, RB, True )


### <a name="centroid_iterator"></a> [Centroid Iterator](#methods)

When running the comparison system, we are looking to compare new records from the deck against the most representative record from each group. This is conceptually similar to the centroid from the K-Means algorithm. This function iterates through the centroids and returns its corresponding record. This method is not just a function of its own. Rather, it is a demonstration of how the process will operate. The function itself is essentially the [Iterator](#iterator) function without the destruct option and with the added input of group number: call fifth group leader, then sixth, etc.


In [10]:
# ----- ITERATE CENTROID RECORDS ----- #

def centroid_iterator( df, grp_no, verb ):
    """Returns centroid record for a specific group number."""
    
    # Speak if necessary
    if verb: print( 'Calling group #{}.'.format(grp_no) )
        
    # Return record for grp_no
    return df.iloc[ Groups.loc[ Groups[ Groups.columns[2] ] == True ].iloc[grp_no][ Groups.columns[1] ] ]
    
# <----- DEMO -----> #
    
# # Iterate through first n group centroids
# for i in range(0, 2):
#     print( list(group_iterator( i, True ).values) )

### <a name="score_similarity"></a> [Score Similarity](#methods)

Now that we have a concise way to readjust the weights with respect to the missing elements in records, we can work on developing a concise function to assess their similarity. This function calls for [weight adjustment](#adjust_weights) as well.


In [11]:
# ----- SCORE SIMILARITY ----- #

def score_similarity( df, RA, RB, verb ):
    """Returns the similarity between two records."""
    
    # Speak if necessary
    if verb: print( 'Scoring record #{} against #{}.'.format(RA.name, RB.name) )
    
    # Instantiate s
    s = np.array([])
    
    # Run comparison feature for feature
    for j in range(0, len(df.columns)):
        s = np.append(s, fuzz.ratio( RA.apply(str)[j], RB.apply(str)[j] ) )

    # Linear product with adjusted weights
    s = np.inner( list(s), adjust_weights( Omega, RA, RB, verb ) )
    if verb: print( 'Similarity: {}'.format(s) )
    
    # Return score
    return s
    
# <----- DEMO -----> #

# # Pull two records
# RA = Dataset.iloc[7].copy()
# RB = Dataset.iloc[8].copy()

# # Score records
# score_similarity( Dataset, RA, RB, True )


### <a name="score_against_groups"></a> [Score Against Groups](#methods)

Here we can call the similarity score function in conjuction with the group and dataframe iterators. We will use the iterator non-destructively with the centroid iterator to demonstrate how a record can be popped off of the deck, assess for missing values, scored with Levenshtein Distance against each existing group.

This function returns a list of scores. These scores represent the similarities between the card and each existing group centroid. The index of the max score links to the group number with which the score is associated. This makes it easier to debug the $s \geq S$ decisions and ensures that the decisions logic is outside te purview of this function.


In [12]:
# ----- SCORE CARD AGAINST CENTROIDS ----- #

def score_against_groups( df, card, verb ):
    """Returns the similarities between a record and each centroid."""

    # Instantiate scores list
    scores = np.array([])

    # Iterate through groups
    for i in range(0, len( Groups.loc[ Groups[ Groups.columns[2] ] == True ] )):

        # Pop ith group centroid record
        GN = centroid_iterator( df, i, verb )

        # Score card against ith group
        s = score_similarity( df, card, GN, verb )
            
        # Append score to scores list
        scores = np.append(scores, s)
        
    # Return scores list
    return scores

# <----- DEMO -----> #

# # Setup
# verb = True

# # Pop card from deck
# card = iterator( Dataset, False, verb )

# # Score card v. centroids
# scores = score_against_groups( Dataset, card, verb )
# list(scores)


### <a name="centroid_assessment"></a> [Centroid Assessment](#methods)

Within the K-means algorithm, the center of a cluster adjusts when new records are assigned. Similarly, we may create individual groups due to uniqueness or incompleteness and then find a more accurate representative for a given identity. Consequently, we must periodically reevaluate which records are centroids.

In the *Representing Group Centers* notes I describe two measurements for determining the centroid of a group. If the number of records in a group is 2 or fewer then the centroid must be the most complete record: the record with the fewest missing values.* If a group has three or more records then we can take the inner similarity between such records and determine which record is most similar to the records within its set. This way we know that we have selected the best representative of the group.

*Additionally, we could find which record has the longest strings, and is therefore the most detailed but this approach has its drawbacks.


In [13]:
# Iterate groups
    # Select a group
        # 1 record
            # Record is set to centroid
        # 2 records
            # Find the most complete record
            # Tie?
            # - YES - #
                # Don't change the record centroid
            # - NO - #
                # Choose the winner as the centroid
        # More than 2 records
            # Find the inner similarities and completeness
            # Select the similarity winner

In [14]:
# ----- CENTROID ASSESSMENT ----- #

def centroid_assessment( df, verb ):
    """Evaluates existing groups to elect new centroids."""
    
    # Speak if necessary
    if verb: print('Reassigning centroids.')

    # Iterate through groups
    for i in range( 0, np.unique( Groups[ Groups.columns[0] ].values ).size - 1 ):

        # Select the ith group of records
        Group = Groups.loc[ Groups[ Groups.columns[0] ] == i ].copy()

        # Only one record?
        if len(Group) == 1:

            # Assign that one record as the centroid
            Groups.at[ Group.loc[ Group[ Group.columns[2] ] == 1 ].index[0], Group.columns[2] ] = 1

        # Two records?
        elif len(Group) == 2:

            # Instantiate missing set
            missing = np.array([])

            # Find missing values for each record
            for r in [0, 1]:
                missing_r = list(np.where( df.iloc[ Group.iloc[r][ Group.columns[1] ] ].isnull().values == True )[0])
                missing = np.append(missing, len(missing_r))

            # If there are missing values, which one is missing the fewest features?
            if sum( missing ) != 0:
                # Find index of the most complete record
                centroid = [ i for i, j in enumerate(missing) if j == min(missing) ]

                # Dethrone the current centroid
                Groups.at[ Group.loc[ Group[ Group.columns[2] ] == 1 ].index[0], Groups.columns[2] ] = 0

                # Assign the most complete record as the centroid
                Groups.at[ Group.iloc[ centroid ].index[0], Groups.columns[2] ] = 1

        # More than 2 records
        elif len(Group) > 2:

            # Instantiate inner similarity set
            inner_similarity = np.zeros(( len(Group) , len(Group) ))

            # Score the similarities of each record
            for rj in range( 0, len(Group) ):
                for ri in range( 0, len(Group) ):
                    inner_similarity[rj, ri] = score_similarity( df, df.iloc[ Group.iloc[rj][ Group.columns[1] ] ],
                                                                df.iloc[ Group.iloc[ri][ Group.columns[1] ] ], False )

            # Take the average of each column
            candidate_score = np.array([])
            for j in range( 0, len(inner_similarity) ):
                candidate_score = np.append( candidate_score, np.average( inner_similarity[:, j] ) )

            # Find the index of the record with the highest commonality
            centroid = [ i for i, j in enumerate(candidate_score) if j == max(candidate_score) ]

            # Dethrone the current centroid
            Groups.at[ Group.loc[ Group[ Group.columns[2] ] == 1 ].index[0], Groups.columns[2] ] = 0

            # Assign the most common record as the centroid
            Groups.at[ Group.iloc[ centroid ].index[0], Groups.columns[2] ] = 1
            
        else:
            print('ERROR: Perverse logic in centroid_assessment.')
            
# <----- DEMO -----> #

# # Call
# print(Groups)
# centroid_assessment( Dataset, False )
# print(Groups)


### <a name="append_to_groups"></a> [Append to Groups](#methods)

This function is called to edit the Groups dataframe. It receives a copy of the Groups dataframe and appends a new entry with the group number, record number and a centroid flag then it returns the altered copy. The function can only be called like so

```
Groups = append_to_groups( Groups, grp_no, rec_no, new, verb )
```

Otherwords changes will not be saved to the dataframe as this function only edits and returns a copy. If the record is being added as a new group then the `new` argument must be set to True so that the new group has a centroid. Otherwise, set the `new` argument to False to preserve the centroid of an existing group.


In [15]:
# ----- APPEND TO GROUPS ----- #

def append_to_groups( df, grp_no, rec_no, new, verb ):
    """Adds a record to the groups dataframe."""
    
    # Speak if necessary
    if verb:
        if new: print( 'Appending record #{} as a new group.'.format(rec_no) )
        if not new: print( 'Appending record #{} to group #{}.'.format(rec_no, grp_no) )
        
    # Configure centroid flag
    if new: centroid = 1
    else: centroid = 0
    
    # Create temp record for append
    Record = pd.DataFrame(data = { df.columns[0] : grp_no, df.columns[1] : rec_no, df.columns[2] : centroid }, index = {0})
    df = df.append( Record, ignore_index = True )
    
    # Return changes
    return df

# <----- DEMO -----> #

# Set group number
grp_no = 1

# Call function
Groups = append_to_groups( Groups, grp_no, 901, True, True )


Appending record #901 as a new group.


In [16]:
# Search for entries within group number
Groups.loc[ Groups[ Groups.columns[0] ] == grp_no ]

,GROUP NUMBER,RECORD NUMBER,CENTROID FLAG
2,1,2,1
11,1,901,1


### <a name="progress"></a> [Progress Bar](#methods)

These functions are designed to support the operation of the progress bar, an ASCII-art graphical visual of the status of the clustering process. The clustering can be arduously slow, necessitating the development of a system to communicate the programs proximity to completion.


In [17]:
# ----- APPEND TO GROUPS ----- #

# Theoretical process with intermediate output
def process(ttime):
    time.sleep( ttime )

# Define progress bar functionality
def update_progress(name, progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = name + ": [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)


## <a name="algorithm"></a> [Full Algorithm](#TOC)
---

Using the methods designed in the previous section, we now compile our work into a single cohesive system of iterating through the dataset, clustering, and returning the final results. The over arching process is described in more detail by the Lucid Chart diagrams.

### <a name="AlgoStages"></a> Algorithm Stages:
---
* [Instantiation](#instantiate)
* [Iteration](#iterate)
* [Pull Group Records](#pull)


### <a name="instantiate"></a> [Instantiation](#AlgoStages)
---

Prior to executing the algorithm, we must take in user preferences and establish the algorithms necessary components.


In [18]:
# ---------- INSTANTIATE ---------- #

# -----> PREFERENCES

# Omega bias preferences
Omega = [ 0.18, 0.15, 0.20, 0.05, 0.05, 0.10, 0.02, 0.25 ]

# Set big S
S = 75

# Verbosity
verb = False

# -----> CREATE COPIES OF THE DATASET

Master = Data.copy()
Deck   = Data.copy()

# -----> CREATE GROUP DATAFRAME

# Template columns
grp_col = [ 'Group Number', 'Record Number', 'Centroid Flag' ]

# Form data for simulation
Groups = pd.DataFrame( columns = grp_col )

# -----> START FIRST GROUP

# Pop from deck
card = iterator( Deck, True, False )

# Append card to Groups
Groups.loc[0] = [ 0, card.name, 1 ]
Groups


,Group Number,Record Number,Centroid Flag
0,0,17,1


### <a name="iterate"></a> [Iteration](#AlgoStages)
---

After having set the stage for the algorithms execution, we can iterate through the deck and proceed with the clustering process.


In [19]:
# ---------- ITERATE THROUGH DECK ---------- #

# Define the number of steps to completion
number_of_elements = len(Deck)

# Main loop
for c in range( 0, len(Deck) ):

    # Pop card from deck
    card = iterator( Deck, True, False )
    
    # Speak if necessary
    if verb: print( 'Card is record #{}'.format(card.name) )

    # Obtain scores
    scores = score_against_groups( Master, card, verb )

    # Take the highest score information
    max_score_ind = np.argmax( scores )

    # Branch based on s > S?
    if scores[ max_score_ind ] >= S:

        # Append record to an existing group
        Groups = append_to_groups( Groups, max_score_ind, card.name, False, verb )

    else:

        # Append record as a new group
        Groups = append_to_groups( Groups, np.max( Groups[ Groups.columns[0] ].values )+1, card.name, True, verb )
        
    # Reassess centroids
    centroid_assessment( Master, verb )
    
    # Call to update progress bar
    if not verb: update_progress('Clustering', c / number_of_elements)
    
# Complete progress bar
if not verb: update_progress('Clustering', 1)

# Print results
Groups


Clustering: [####################] 100.0%


,Group Number,Record Number,Centroid Flag
0,0,17,0
1,0,16,1
2,1,15,0
3,1,14,1
4,1,13,0
5,2,12,1
6,3,11,1
7,4,10,1
8,5,9,1
9,6,8,1


In [20]:
# Harness a specific group number
Groups.loc[ Groups[ Groups.columns[0] ] == 1 ]

,Group Number,Record Number,Centroid Flag
2,1,15,0
3,1,14,1
4,1,13,0


### <a name="pull"></a> [Pull Group Records](#AlgoStages)
---

With the clustering hierarchy in place, we can now append the group labeling to the original data to see if our human intuition agrees with the results.


In [21]:
Master.join( pd.DataFrame( { Groups.columns[0] : list( reversed( Groups[ Groups.columns[0] ].values.tolist() )) } ) )

,FIRST,MIDDLE,LAST,ADDRESS,CITY,STATE,ZIP,SSN,Group Number
0,HARVEY,B,MILK,1930 RECRUIT YOU AVE,SAN FRANCISCO,CA,18896.0,314159265.0,12
1,HARVEY,B,MILK,1930 RECRUIT YOU AVE,SF,CA,18896.0,314159265.0,12
2,H,B,MLLK,1930 RECRUIT YOU AVE,SF,CA,NaN,314159265.0,11
3,H,NaN,NaN,1930 RECRUIT U,NaN,NaN,18896.0,NaN,11
4,RICK,P,SANCHEZ,EARTH C-137,TACOMA,WA,98402.0,929601596.0,10
5,PYOTR,I,TCHAIKOVSKY,1840 N ST PETER,VOTKINSK,GA,11893.0,155633999.0,9
6,RUPERT,L,DONLEY,518 EVEREST BLVD,UDEMY,FL,12358.0,123789235.0,8
7,ELEANOR,P,TWINE,7878 SEVEN EIGHT,SMALL TOWN,LA,26916.0,148978105.0,7
8,MICHELL,M,MENENDEZ,58 CLOUDLESS SKY WAY,NaN,NaN,32156.0,203254687.0,6
9,CHARLES,F,KANE,794 BROKEN HOME AVE,ROSEBUD,PA,56465.0,489324789.0,5


*Notebook created by Austin Dial on 6/12/2019* <br>
*Proof of Concept completed by Austin Dial on 6/21/2019* <br>
*Development of Methods and Algorithm completed by Austin Dial on 7/1/2019*
*Test data was updated by Austin Dial on 7/12/2019*